In [1]:
import gradio as gr
from typing import Optional, List, Any
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, TextIteratorStreamer, AutoProcessor
# from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.outputs.generation import GenerationChunk
from torch import device
import time
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains.history_aware_retriever import create_history_aware_retriever

from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain


from threading import Thread
import warnings
warnings.filterwarnings("ignore")

/root/miniconda3/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/root/miniconda3/lib/python3.8/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issu

# 该部分代码实现
## 1.历史记录聊天
## 2.实体提取
## 3.意图识别
## 4.主体模型qwen2.5-7B-instruct

In [2]:
# 构建模型
class QianWenLLM(LLM):
    # 基于本地的QianWen7B-Chat模型自定义LLM类
    tokenizer: AutoTokenizer = None
    model: AutoModelForCausalLM = None
    processor: AutoProcessor = None
    
    def __init__(self, model_dir: str):
        # 从本地加载模型
        super().__init__()
        print('正从本地加载模型。。。。。')

        self.tokenizer = AutoTokenizer.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            trust_remote_code=True,
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            device_map='auto',
            trust_remote_code=True,
            # torch_dtype=torch.bfloat16,
            # temperature=0.1
            )
        self.model = self.model.eval()
        self.model.generation_config = GenerationConfig.from_pretrained(
            model_dir,
            trust_remote_code=True
        )
        # 可指定不同的生成长度、top_p等相关超参
        self.processor = AutoProcessor.from_pretrained(model_dir)
        print('模型加载完成！')

    def _call(self, messages,
              stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs: Any):
        # response, history = self.model.chat(self.tokenizer, prompt, history=[])
        if type(messages) == str:
            system_info =  messages.split('System:')[-1].split('Human:')[0].strip()
            human_info =  messages.split('Human:')[-1].strip()
            re_prompt = [
                    {'role':'system', 'content': system_info},
                    {'role':'user', 'content': human_info}
                    ]
            messages = re_prompt
        # print('打印messages内容：', messages)
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = self.tokenizer([text], return_tensors='pt').to('cpu')
        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=512
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        return response
        
    @property
    def _llm_type(self) -> str:
        return "QwenLM"

In [3]:
# 构建改写问题prompt

# 请根据聊天历史和最后用户的问题，改写用户最终剔除的问题。
# 你只需要改写用户最终的问题，请不要回答问题。
# 没有聊天历史则将用户问题直接返回，有聊天历史则改写。
    
def contextualize_question_prompt():
    system_prompt = """\
    根据聊天历史和输入文本，改写输入文本。请根据工作流程和工作要求改写文本。

    工作流程：
    1-判断聊天历史是否为空。内容为空直接返回输入文本，否则继续以下工作流程。
    2-聊天历史存在内容，请根据聊天历史和输入文本内容相关程度，改写输入文本。如果相关程度低则直接输出文本。
    3-请思考改写后的输入文本内容，是否突出用户文本意图，否则重新根据用户问题改写。请注意如果是打招呼等日常用语也直接返回用户问题。
    4-请不要回答输入的文本，输出改写的文本内容。

    工作要求：
    1-你只需要改写文本。
    2-聊天历史为空则将输入文本直接返回，有聊天历史则改写。    
    """
    contextualize_question_prompt = ChatPromptTemplate(
        [
        ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', '{input}')
        ]
    )
    return contextualize_question_prompt

def answer_prompt():
    # 构建正常问答prompt
    system_prompt = """\
    使用上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。
    {context}
    """
    # 问题: {question}
    # 有用的回答:
    
    qa_prompt = ChatPromptTemplate(
    [
    ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', '{input}')
    ]
    )
    return qa_prompt

store = {}
# 构造存储函数
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [4]:
def init_model():
    # 初始化模型
    # model_cache_path = r'autodl-tmp/Qwen/Qwen-7B-Chat'
    model_cache_path = r'autodl-tmp/Qwen/Qwen2.5-7B-Instruct'
    llm = QianWenLLM(model_dir=model_cache_path)
    return llm

In [5]:
def init_db():
    persist_directory = r'vectordb/chroma'
    embeddings_model_cache_path = r'autodl-tmp/embedding_model/Ceceliachenen/paraphrase-multilingual-MiniLM-L12-v2'
    # 加载词向量模型
    embeddings = HuggingFaceEmbeddings(
        model_name=embeddings_model_cache_path)
    # 加载缓存知识库
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
    )
    return vectordb

In [6]:
def intention_recognition(question: str, llm):
    template = f"""
        你是一名实体提取和意图识别分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。
        
        任务：
        1-判断用户问题是否存在实体。
        2-抽取用户问题所有实体。
        3-根据实体与给出的意图标签进行判定该用户问题的意图。
        
        意图标签：政策知识，日常知识，招中标知识
        
        工作流程：
        1.-先判断是否存在实体，不存在实体则直接根据不存在实体输出格式输出，存在实体则继续以下工作流程，并通过存在实体输出格式输出。
        2-实体提取：请从用户问题中提取出所有实体。
        3-意图分类：请根据第2点提取的实体以及意图标签，进行意图识别并分类用户问题。
        
        不存在实体输出格式：
            实体提取:[]
            意图分类:日常知识
            
        存在实体输出格式：
            实体提取：[实体1, 实体2, ...]
            意图分类：意图标签
            
        有用的回答：
       """
          # 4. 意图分类：根据第3点的意图识别结果分类，意图标签：[政策知识，日常知识，招中标知识]。       2.意图识别：[意图1, 意图2, ...]。
    # ，例如日常、政策、设备、建筑、维修等
    messages = [
            {'role':'system', 'content': template},
            {'role':'user', 'content':question}]

    response = llm._call(messages)
    print('实体+意图模型结果：', response)
    target = response.split('意图分类：')[-1]
    # print('模型实体+意图结果：', response)
    # print('意图分类:', target)
    return target
        

In [7]:
def qa_chain(rag_chain, get_session_history):
    """
    构建问答链
    :param persist_directory: 知识库本地保存路径，这里初始化政策信息知识库
    :return: 返回调用LLM回答
    """
    # 包装
    conversational_rag_chain = RunnableWithMessageHistory(
        runnable=rag_chain,
        get_session_history=get_session_history,
        input_messages_key='input',
        history_messages_key='chat_history',
        output_messages_key='answer'
    )

    return conversational_rag_chain


In [8]:
def contextualize_question_chain(llm):
    # 根据历史检索器原理，我们这里只要llm和改写prompt和历史记录就行
    question_prompt = contextualize_question_prompt()
    contextualize_question_chain = RunnableWithMessageHistory(
        question_prompt | llm,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history"
    )
    return contextualize_question_chain

In [9]:
# 链接前面的函数
class Model_center():
    """
      存储问答 Chain 的对象
    """

    def __init__(self, ):
        print('初始化模型+知识库。。。。')
        self.model = init_model()
        self.db = init_db()
         # 因为历史记录问题进行问题改写再传到模型
        self.question_prompt = contextualize_question_prompt()
        # 历史对话巡回器
        self.history_aware_retriever = create_history_aware_retriever(self.model, 
                                                                self.db.as_retriever(), 
                                                                 self.question_prompt)
        self.qa_prompt = answer_prompt()
        self.qa_chain = create_stuff_documents_chain(self.model, self.qa_prompt)
        self.rag_chain = create_retrieval_chain(self.history_aware_retriever, self.qa_chain)
        

    def qa_chain_self_answer(self, question: str, chat_history: list = []):
        # print('调用问答链')
        # print('打印用户问题', question)
        if question == None or len(question) < 1:
            print('问答为空。。。。')
            return '', chat_history
        print('实体提取+意图识别。。。。')

        # for question in sentence:
        # try
        # print('用户输入问题：', question)
        target = intention_recognition(question, self.model)
        print('意图识别成功。。。。')
        # except:
            # print('实体提取+意图识别步骤失败。。。。')


        if target == '招中标知识' or target == '政策知识':
            print('调用检索问答链。。。。')
            # print('知识库:', target)
            # 带langchain实力对话共能
            response = qa_chain(
                self.rag_chain,
                get_session_history).invoke(
                    {'input': question},
                    config={'configurable':{'session_id':'test123'}}
                    )
            print('调用检索问答链输出response：', response['context'][0].page_content)

        else:
            
            print('调用简单模型。。。。')
            
            # 改写问题
            
            # 将改写的新问题输入进行回答
            new_question = contextualize_question_chain(self.model).invoke( 
                {'input': question},
                config={'configurable':{'session_id':'test123'}}
                    )
            print('用户问题：', question)
            print('根据历史聊天记录改写用户问题：', new_question)

            template = """\
                        请回答用户的问题，如果你不知道答案，就说你不知道，不要试图编造答案。
                    """
            messages = [
                {'role':'system', 'content': template},
                {'role':'user', 'content':new_question}
                ]
            print('改写前的问题：', question, '改写后的问题：', new_question)
            response = self.model._call(messages)
            store['test123'].add_user_message(question)
            store['test123'].add_ai_message(response)
        print(f'意图分类：{target}，测试问题：{question}，返回结果：{response}')

    # print('简单模型输出格式：', type(response))
        # 结果调用下流式输出    target == '日常知识':
        
        
        # 检索问答链+历史聊天组件
        # response = self.qa_chain.invoke({'query': question, 'chat_history': chat_history})['answer']

        chat_history.append([question, response])
        # print(chat_history)
        return '', chat_history
                
        # except Exception as e:
        #     print('问答链报错', e)
        #     return e, chat_history

    def clear_history(self):
        self.qa_chain.clear_history()

In [10]:
model_center = Model_center()
def update_chatbot(question, chat_history):
    for char in model_center.qa_chain_self_answer(question, chat_history):
        gr.update(value=chat_history)
        chat_history.append((question, char))
    return chat_history
# def demo():
block = gr.Blocks()
with block as demo:
    with gr.Row(equal_height=True):  # 水平排列子组件
        with gr.Column(scale=15):  # 垂直排列子组件
            gr.Markdown("""<h1><center>QwenLM7B-Chat</center></h1><center>科大讯飞实践-招中标政策智能问答助手</center>""")

    with gr.Row():
        with gr.Column(scale=4):
            # 创建聊天界面的组件。height=450 参数设置了聊天界面的高度为 450 像素。
            # show_copy_button=True参数表示在聊天界面中显示一个复制按钮，允许用户复制聊天内容
            chatbot = gr.Chatbot(height=450, show_copy_button=True)
            # 创建一个文本框组件，用于输入 prompts。
            msg = gr.Textbox(label='Prompt/问题')

            with gr.Row():
                # 创建提交按钮
                db_wo_his_btn = gr.Button('Chat')
            with gr.Row():
                # 创建一个清除按钮，用于清除聊天机器人组件的内容。
                clear_btn = gr.ClearButton(components=[chatbot], value='Clear console')

        # 设置按钮的点击事件。当点击时，调用上面定义的 qa_chain_self_answer 函数，并传入用户的消息和聊天历史记录，然后更新文本框和聊天机器人组件。
        print('进度1')
        # 设置流式输出
        def bot(question, history):
            # print('bot_question',question)
            # print('bot_history',history)
            curr, response = model_center.qa_chain_self_answer(question, history)
            # print('response', response)
            # print('curr', curr)
            history = response
            bot_message = history[-1][1]
            # print('bot_message', bot_message)
            history[-1][1] = ''
            for character in bot_message:
                history[-1][1] += character
                # print(f'累计中：{history}')
                time.sleep(0.1)
                yield '', history
        
        db_wo_his_btn.click(bot, inputs=[msg, chatbot], outputs=[msg, chatbot])

        print('进度2')
        # 点击后清空后端存储的聊天记录
        clear_btn.click(model_center.clear_history)

    # 填写注意事项
    gr.Markdown(
        """
        提醒：<br>
        1. 初始化数据库实践可能较长，请耐心等待。
        2. 使用中如果出现异常，将会在文本输入框进行展示，请不要惊慌。 <br>
        """
    )
# gr.close_all()
# 直接启动
demo.queue()
demo.launch(server_name='127.0.0.1', server_port=6010)

初始化模型+知识库。。。。
正从本地加载模型。。。。。


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


模型加载完成！
进度1
进度2
Running on local URL:  http://127.0.0.1:6010

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
实体提取+意图识别。。。。
实体+意图模型结果： 实体提取：投标文件，招标人

意图分类：招中标知识
意图识别成功。。。。
调用检索问答链。。。。
调用检索问答链输出response： 络或者其他媒介发布的招标文件与书面招标文件具有同等法律效力，出现不一致时以书面招标文件为准，国家另有规定的除外。对招标文件或者资格预审文件的收费应当限于补偿印刷、邮寄的成本支出，不得以营利为目的。对于所附的设计文件，招标人可以向投标人酌收押金；对于开标后投标人退还设计文件的，招标人应当向投标人退还押金。招标文件或者资格预审文件售出后，不予退还。除不可抗力原因外，招标人在发布招标公告、发出投标邀请书后或者售出招标文件或资格预审文件后不得终止招标。第十六条招标人可以根据招标项目本身的特点和需要，要求潜在投标人或者投标人提供满足其资格要求的文件，对潜在投标人或者投标人进行资格审查；国家对潜在投标人或者投标人的资格条件有规定的，依照其规定。第十七条资格审查分为资格预审和资格后审。资格预审，是指在投标前对潜在投标人进行的资格审查。资格后审，是指在开标后对投标人进行的资格审查。进行资格预审的，一般不再进行资格后审，但招标文件另有规定的除外。第十八条采取资格预审的，招标人应当发布资格预审公告。资格预审公告适用本办法第十三条、第十四条有关招标公告的规定。采取资格预审的，招标人应当在资格预审文件中载明资格预审的条件、标准和方法；采取资格后审的，招标人应当在招标文件中载明对投标人资格要求的条件、标准和方法。招标人不得改变载明的资格条件或者以没有载明的资格条件对潜在投标人或者投标人进行资格审查。第十九条经资格预审后，招标人应当向资格预审合格的潜在投标人发出资格预审合格通知书，告知获取招标文件的时间、地点和方法，并同时向资格预审不合格的潜在投标人告知资格预审结果。资格预审不合格的潜在投标人不得参加投标。经资格后审不合格的投标人的投标应予否决。第二十条资格审查应主要审查潜在投标人或者投标人是否符合下列条件：（一）具有独立订立合同的权利；（二）具有履行合同的能力，包括专业、技术资格和能力，资金、

In [ ]:
sentence = [
            '你好',
            '你是谁？',
            '上海联合产权交易所有限公司物权交易操作指引(沪联产交〔2020〕26号)',
            '周杰伦是谁？',
            '产权交易争议调解',
            '2023年7月，中国航天科技集团公司五院研制的长征十二号运载火箭在酒泉卫星发射中心成功发射，将一颗北斗导航卫星送入预定轨道。',
            '周杰伦的歌曲《稻香》是谁写的？',
            '项目名称为ABC，中标金额为500万元，供应商为XYZ公司。']

In [ ]:
template = f"""
        你是一名实体提取和意图识别分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。
        
        任务：
        1-判断输入文本是否存在实体。
        2-抽取输入文本所有实体。
        3-根据实体与给出的意图标签进行判定该输入文本的意图。
        
        意图标签：政策知识，日常知识，招中标知识
        
        工作流程：
        1.-先判断是否存在实体，不存在实体则直接根据不存在实体输出格式输出，存在实体则继续以下工作流程，并通过存在实体输出格式输出。
        2-实体提取：请从输入的文本中提取出所有实体。
        3-意图分类：请根据第2点提取的实体以及意图标签，进行意图识别并分类输入文本。
        
        输入文本：{question}
        不存在实体输出格式：
            实体提取:[]
            意图分类:日常知识
        存在实体输出格式：
            实体提取：[实体1, 实体2, ...]
            意图分类：意图标签

        有用的回答：
       """

In [10]:
question_prompt = f"""\
    根据聊天历史和用户问题，改写用户问题。请根据工作流程和工作要求改写问题。

    工作流程：
    1-判断聊天历史是否为空。内容为空直接返回用户的问题，否则继续以下工作流程。
    2-聊天历史存在内容，请根据聊天历史和用户的问题，改写用户提出的问题。
    3-请思考改写后的问题内容，是否突出用户问题意图，否则重新根据用户问题改写。请注意如果是打招呼等日常用语也直接返回用户问题。
    4-请不要回答用户的问题，输出改写的问题。

    工作要求：
    1-你只需要改写用户的问题。
    2-聊天历史为空则将用户问题直接返回，有聊天历史则改写。                         

    聊天历史：{chat_history_}
    """

NameError: name 'chat_history_' is not defined

In [11]:
def history():
    MessagesPlaceholder('chat_history')

NameError: name 'response' is not defined

Mess